In [37]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

no display found. Using non-interactive Agg backend


In [20]:
Takyo = pd.read_csv("/Users/bradleybeck/Documents/MDA 620/Tayko.csv")
predictors = ['Freq', 'last_update_days_ago', 'Web order', 'Gender=male', 'Address_is_res']
outcome = 'Spending'

In [45]:
Takyo.head()

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,128
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0
2,3,1,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127
3,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0
4,5,1,0,1,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0


In [21]:
X = pd.get_dummies(Takyo[predictors],drop_first=True)
X

,Freq,last_update_days_ago,Web order,Gender=male,Address_is_res
0,2,3662,1,0,1
1,0,2900,1,1,0
2,2,3883,0,0,0
3,1,829,0,1,0
4,1,869,0,0,0
...,...,...,...,...,...
1995,1,1701,1,0,0
1996,1,2633,1,1,0
1997,0,3394,0,0,0
1998,1,253,0,1,1


In [26]:
y = Takyo[outcome]

In [27]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3,random_state=1)

In [29]:
Takyo_lm = LinearRegression()
Takyo_lm.fit(train_X, train_y)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': Takyo_lm.coef_}))

              Predictor  coefficient
0                  Freq    92.649820
1  last_update_days_ago    -0.008939
2             Web order    17.689162
3           Gender=male    -7.814974
4        Address_is_res   -77.645643


In [31]:
Takyo_lm_pred = Takyo_lm.predict(valid_X)
result = pd.DataFrame({'Predicted': Takyo_lm_pred, 'Actual': valid_y,
                       'Residual': valid_y - Takyo_lm_pred})
print(result.head(20))
regressionSummary(valid_y, Takyo_lm_pred)

       Predicted  Actual     Residual
674    89.704569       0   -89.704569
1699  201.006344     184   -17.006344
1282   53.506990       0   -53.506990
1315  835.536729    1289   453.463271
1210   -5.129154       0     5.129154
1636   87.595061       0   -87.595061
613    61.140548       0   -61.140548
447   248.370924    1255  1006.629076
1131   68.875075       0   -68.875075
808    69.554408       0   -69.554408
1496   -7.322517       0     7.322517
1468  194.615251     411   216.384749
1682  -14.505426       0    14.505426
1149  -30.821001       0    30.821001
442    63.887340       0   -63.887340
1813    0.361716     173   172.638284
654   -42.530556       0    42.530556
1264  -34.715582       0    34.715582
858    81.871476       0   -81.871476
1482   55.768454       0   -55.768454

Regression statistics

               Mean Error (ME) : 8.0998
Root Mean Squared Error (RMSE) : 147.6752
     Mean Absolute Error (MAE) : 88.0750


In [42]:
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def Takyo_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

allVariables = train_X.columns
best_model, best_variables = backward_elimination(allVariables, train_model, 
                                 Takyo_model, verbose=True)

print(best_variables)

regressionSummary(valid_y, best_model.predict(valid_X[best_variables]))

Variables: Freq, last_update_days_ago, Web order, Gender=male, Address_is_res
Start: score=17441.47
Step: score=17440.89, remove Gender=male
Step: score=17440.89, remove None
['Freq', 'last_update_days_ago', 'Web order', 'Address_is_res']

Regression statistics

               Mean Error (ME) : 8.0327
Root Mean Squared Error (RMSE) : 147.4527
     Mean Absolute Error (MAE) : 87.8026


In [43]:
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def Takyo_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, Takyo_model,
                                 verbose=True)
print(best_variables)

Variables: Freq, last_update_days_ago, Web order, Gender=male, Address_is_res
Start: score=18535.28, constant
Step: score=17534.41, add Freq
Step: score=17452.04, add Address_is_res
Step: score=17445.85, add last_update_days_ago
Step: score=17440.89, add Web order
Step: score=17440.89, add None
['Freq', 'Address_is_res', 'last_update_days_ago', 'Web order']


In [44]:
best_model, best_variables = stepwise_selection(train_X.columns, train_model, Takyo_model, 
                                 verbose=True)
print(best_variables)

Variables: Freq, last_update_days_ago, Web order, Gender=male, Address_is_res
Start: score=18535.28, constant
Step: score=17534.41, add Freq
Step: score=17452.04, add Address_is_res
Step: score=17445.85, add last_update_days_ago
Step: score=17440.89, add Web order
Step: score=17440.89, unchanged None
['Freq', 'Address_is_res', 'last_update_days_ago', 'Web order']
